# Football Data Exploration

This notebook explores the football match dataset to understand its structure and characteristics.


In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append("src")

In [ ]:
from spark_session import create_spark_session

spark = create_spark_session()

In [3]:
from data_loader import load_data

# Load data (using local path for development)
(matches_df, elo_df), _ = load_data(spark, local_path=True)

In [4]:
# Basic statistics
print(f"Number of countries: {elo_df.select('Country').distinct().count()}")
print(f"Number of Elo ratings: {elo_df.count()}")
print(f"Number of clubs with recorded Elo ratings: {elo_df.select('Club').distinct().count()}")
print("")
print(f"Number of league keys: {matches_df.select('League').distinct().count()}")
print(f"Number of matches: {matches_df.count()}")
print(
    f"Number of teams with recorded matches: {matches_df.select('HomeTeam').union(matches_df.select('AwayTeam')).distinct().count()}"
)
print("")
# Sample data
elo_df.show(5)
matches_df.show(2)

Number of countries: 19
Number of Elo ratings: 242591
Number of clubs with recorded Elo ratings: 895

Number of league keys: 38
Number of matches: 228377
Number of teams with recorded matches: 1217

+----------+--------+-------+-------+
|      Date|    Club|Country|    Elo|
+----------+--------+-------+-------+
|2000-07-01|  Aachen|    GER| 1453.6|
|2000-07-01| Aalborg|    DEN|1482.61|
|2000-07-01|   Aalst|    BEL|1337.53|
|2000-07-01|  Aarhus|    DEN|1381.46|
|2000-07-01|Aberdeen|    SCO|1360.43|
+----------+--------+-------+-------+
only showing top 5 rows

+------+----------+---------+---------+----------+-------+-------+---------+---------+---------+---------+------+------+--------+------+------+--------+---------+---------+----------+----------+---------+---------+-----------+-----------+----------+----------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+---------+----------+---------+---------+---------+
|League|      Date|MatchTime| HomeTeam|  Aw

We seem to have all of the Elo ratings in the `matches_df`, so maybe the `elo_df` is pointless for our purpose. According to the description on Kaggle, the Elo ratings in the `elo_df` are taken from [ClubElo](httsp://www.clubelo.com/), while match results and statistics provided in the table are taken from <https://football-data.co.uk>. It just mentions the Elo ratings in the matches dataset are the "most recent Elo ratings", so we can assume that the Elo ratings are in both cases from the same source. **Since we just want to make predictions for game outcomes, we can decide that only the MATCHES.csv dataset is relevant to us.**

As we can see, the number of teams/clubs is much greater than the expected number (~500). Hm.


In [5]:
from utils import count_nulls

print("\nMissing values in MATCHES.csv:")
count_nulls(matches_df).show()


Missing values in MATCHES.csv:
+------+----+---------+--------+--------+-------+-------+---------+---------+---------+---------+------+------+--------+------+------+--------+---------+---------+----------+----------+---------+---------+-----------+-----------+----------+----------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+---------+----------+---------+---------+---------+
|League|Date|MatchTime|HomeTeam|AwayTeam|HomeElo|AwayElo|Form3Home|Form5Home|Form3Away|Form5Away|FTHome|FTAway|FTResult|HTHome|HTAway|HTResult|HomeShots|AwayShots|HomeTarget|AwayTarget|HomeFouls|AwayFouls|HomeCorners|AwayCorners|HomeYellow|AwayYellow|HomeRed|AwayRed|OddHome|OddDraw|OddAway|MaxHome|MaxDraw|MaxAway|Over25|Under25|MaxOver25|MaxUnder25|HandiSize|HandiHome|HandiAway|
+------+----+---------+--------+--------+-------+-------+---------+---------+---------+---------+------+------+--------+------+------+--------+---------+---------+----------+----------+---------+---------

If we want to use as many rows as possible, we should only try to predict "FTResult", "FTHome", or "FTAway". To make our job less complex, **we will decide to just try to predict "FTResult"**.


In [6]:
spark.stop()